In [1]:
!pip3 install pymisp pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.1 MB/s eta 0:00:00


In [3]:
from Crypto.Cipher import AES
import base64
import hashlib
from Crypto.Random import get_random_bytes

def encrypt(plaintext, key):
    key = hashlib.sha256(key.encode()).digest()
    cipher = AES.new(key, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(plaintext.encode('utf-8'))
    encrypted_data = cipher.nonce + ciphertext
    return base64.b64encode(encrypted_data).decode('utf-8')

plaintext = "MISP_API_KEY"
key = "PROVIDED_KEY_BY_AIS3_COURSE(This is not real key)"

# encrypted_text = encrypt(plaintext, key)
# print("Encrypted Text:", encrypted_text)

In [ ]:
from Crypto.Cipher import AES
import base64
import hashlib

def decrypt(encrypted_text, key):
    key = hashlib.sha256(key.encode()).digest()
    encrypted_text = base64.b64decode(encrypted_text)
    nonce = encrypted_text[:16]
    ciphertext = encrypted_text[16:]
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    plaintext = cipher.decrypt(ciphertext)
    return plaintext.decode('utf-8')


key = "PROVIDED_KEY_BY_AIS3_COURSE(This is not real key)"
encrypted_text = "OpR/RJfaK7psxkK9LJPyqT/XbCLOAISR7ryGdSN4kjEStWwyCztV+Rnq0mlx4JbcZiAVdOwEfac="
decrypted_text = decrypt(encrypted_text, key)
print("Decrypted Text:", decrypted_text)

Decrypted Text: rqSYiiBDnOAzT2UzQ4iXRALW22zJvlgImu8sGZg2


In [11]:
# 請註冊一個 ABuseIPDB 的 API Key
# https://www.abuseipdb.com/account
# 然後填在下面 API_KEY

import requests
import random

API_KEY = 'YOUR_API_KEY'

url = 'https://api.abuseipdb.com/api/v2/blacklist'
headers = {
    'Key': API_KEY,
    'Accept': 'application/json'
}
params = {
    'confidenceMinimum': 90,
    'limit': 100
}

response = requests.get(url, headers=headers, params=params)
data = response.json()

# 隨機取 2 筆 IP
ips = data['data']
random_ips = random.sample(ips, 2)

for ip in random_ips:
    print(ip['ipAddress'])


144.126.207.244
87.106.188.106


In [12]:
from pymisp import MISPEvent,MISPOrganisation, PyMISP
from dateutil.parser import parser
import json
from pymisp.tools import feed_meta_generator
from io import BytesIO

In [13]:
make_feed = False
misp = None

In [14]:
if make_feed:
    org = MISPOrganisation()
    org.name = "ais3.sectools.tw"
    org.uuid = "d2ded987-c2f1-4b12-bf1d-ba9872d0d452"
else:
    url = "https://misp.sectools.tw"
    key = decrypted_text
    misp = PyMISP(url,key,ssl=False)

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [15]:
from datetime import datetime
from pathlib import Path
blacklist_ip = ip['ipAddress']
nick_name = "PLEASE_TYPE_YOUR_NICK_NAME"
for p in blacklist_ip:
    event = MISPEvent()
    event.info = f"[{str(p)}] is suspected of DDoS, found by {str(nick_name)}"
    event.date = datetime.now()
    event.distribution = 3
    event.add_tag('tlp:white')
    if make_feed:
        event.org = org
    else:
        e = misp.search(eventinfo=event.info, metadata=True, pythonify=True)
        if e:
            # Already added.
            continue

    obj_ip = event.add_object(name='domain-ip', standalone=False)
    obj_ip.add_attribute('ip', p)
    obj_ip.add_attribute('first-seen', datetime.now() )
    obj_ip.add_reference(event.uuid, 'related-to',"ais3_friend")

    if make_feed:
        with (Path('output') / f'{event.uuid}.json').open('w') as _w:
            json.dump(event.to_feed(), _w)
    else:
        misp.add_event(event)
    print(event.to_json(indent=2))
if make_feed:
    feed_meta_generator(Path('output'))

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{
  "uuid": "95c8f4f6-9170-4262-89cf-0fa5369c1db1",
  "Object": [
    {
      "name": "domain-ip",
      "meta-category": "network",
      "template_uuid": "43b3b146-77eb-4931-b4cc-b66c60f28734",
      "description": "A domain/hostname and IP address seen as a tuple in a specific time frame.",
      "template_version": "11",
      "uuid": "79ddbc2e-028f-4abd-b0b2-5d7108b347dd",
      "ObjectReference": [
        {
          "uuid": "9fc70421-164f-48b2-971b-f4dbe2472b13",
          "object_uuid": "79ddbc2e-028f-4abd-b0b2-5d7108b347dd",
          "referenced_uuid": "95c8f4f6-9170-4262-89cf-0fa5369c1db1",
          "relationship_type": "related-to",
          "comment": "ais3_friend"
        }
      ],
      "Attribute": [
        {
          "uuid": "c7c78169-e5fe-4af6-8f29-bc841ebd7840",
          "object_relation": "ip",
          "value": "8",
          "type": "ip-dst",
          "category": "Network activity",
          "disable_correlation": false,
          "to_ids": true
        

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{
  "uuid": "01135f6a-be09-4b7d-acc7-84853836d448",
  "Object": [
    {
      "name": "domain-ip",
      "meta-category": "network",
      "template_uuid": "43b3b146-77eb-4931-b4cc-b66c60f28734",
      "description": "A domain/hostname and IP address seen as a tuple in a specific time frame.",
      "template_version": "11",
      "uuid": "edca2d51-0939-447b-8d1d-6c8e3b185b4c",
      "ObjectReference": [
        {
          "uuid": "c133e857-1432-4ed4-921f-5dba5d80aa15",
          "object_uuid": "edca2d51-0939-447b-8d1d-6c8e3b185b4c",
          "referenced_uuid": "01135f6a-be09-4b7d-acc7-84853836d448",
          "relationship_type": "related-to",
          "comment": "ais3_friend"
        }
      ],
      "Attribute": [
        {
          "uuid": "e15feebd-4121-4ef5-b361-67410c9c64dd",
          "object_relation": "ip",
          "value": "7",
          "type": "ip-dst",
          "category": "Network activity",
          "disable_correlation": false,
          "to_ids": true
        

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{
  "uuid": "017188f6-1bdb-47f8-9868-5bec95040cd3",
  "Object": [
    {
      "name": "domain-ip",
      "meta-category": "network",
      "template_uuid": "43b3b146-77eb-4931-b4cc-b66c60f28734",
      "description": "A domain/hostname and IP address seen as a tuple in a specific time frame.",
      "template_version": "11",
      "uuid": "231286c6-abb0-4964-9845-8ee65a1fc0c0",
      "ObjectReference": [
        {
          "uuid": "2910bd96-9038-4fda-b194-6498227530ce",
          "object_uuid": "231286c6-abb0-4964-9845-8ee65a1fc0c0",
          "referenced_uuid": "017188f6-1bdb-47f8-9868-5bec95040cd3",
          "relationship_type": "related-to",
          "comment": "ais3_friend"
        }
      ],
      "Attribute": [
        {
          "uuid": "8e2a6a25-a8ad-437f-8d5a-eb9f7f7af571",
          "object_relation": "ip",
          "value": "0",
          "type": "ip-dst",
          "category": "Network activity",
          "disable_correlation": false,
          "to_ids": true
        

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{
  "uuid": "a0918f8d-e07a-446e-9210-66618581f91c",
  "Object": [
    {
      "name": "domain-ip",
      "meta-category": "network",
      "template_uuid": "43b3b146-77eb-4931-b4cc-b66c60f28734",
      "description": "A domain/hostname and IP address seen as a tuple in a specific time frame.",
      "template_version": "11",
      "uuid": "223fe539-682d-4344-959a-ae35de42f05b",
      "ObjectReference": [
        {
          "uuid": "c225c5b2-e1ff-41db-85bc-f33d6bcabf68",
          "object_uuid": "223fe539-682d-4344-959a-ae35de42f05b",
          "referenced_uuid": "a0918f8d-e07a-446e-9210-66618581f91c",
          "relationship_type": "related-to",
          "comment": "ais3_friend"
        }
      ],
      "Attribute": [
        {
          "uuid": "9925373a-5ec1-4898-87bf-216d7cdea98c",
          "object_relation": "ip",
          "value": "6",
          "type": "ip-dst",
          "category": "Network activity",
          "disable_correlation": false,
          "to_ids": true
        

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.p